In [1]:
from torch import load
import numpy as np

In [2]:
# If you haven't trained a model yet, please go to the file 'baryons_train.ipynb' first!
model = load('../models/test_200.pt')
model.eval()

BaryonModel(
  (linear_in): Linear(in_features=141, out_features=141, bias=True)
  (linear_1): Linear(in_features=141, out_features=32, bias=True)
  (linear_3): Linear(in_features=32, out_features=141, bias=True)
  (leaky): LeakyReLU(negative_slope=0.01)
  (sigmoid): Sigmoid()
)

In [3]:
from hydroml.libs.dataset import Dataset
from torch.utils.data import DataLoader

net_Baryons = Dataset(np.fromfile('../datasets/training/3DAuAu200_minimumbias_BG16_tune17_netBaryon.dat', dtype=np.float32), 141)

data_loader = DataLoader(
    dataset=net_Baryons,
    shuffle=True,
)

In [4]:
from hydroml.utils.plot import plot_cc_graph
from hydroml.utils.moments import Moments
from hydroml.utils.trim import trim

finals = []
models = []
bound_1 = -1
bound_2 = 1

import numpy as np

for i, data in enumerate(data_loader):

    protons = model(data.flatten())

    protons = protons.detach().numpy()

    baryon_eta_trim, baryon_trim = trim( net_Baryons.eta, data.flatten(), bound_1, bound_2  )
    _, output_trim = trim( net_Baryons.eta, protons, bound_1, bound_2  )

    finals.append( np.trapz( baryon_trim, x=baryon_eta_trim ) )
    models.append( np.trapz( output_trim, x=baryon_eta_trim ) )

finals_moments = Moments(finals)
models_moments = Moments(models)

print("Values for actual final state")
print(finals_moments)

print()

print("Values for model output final state")
print(models_moments)

plot_cc_graph(np.array(finals), np.array(models), "Net Protons")

AttributeError: 'BaryonModel' object has no attribute 'linear_2'